# VISUALIZATION OF AIRLINE ROUTES AND AIRPORTS

I analyzed and visualized airline routes between **airports** using **NetworkX** and **Plotly**. Airports were taken as **nodes**, and flight routes between them were considered **edges**. Using the **airports dataset**, I extracted Indian airports, and from the **routes dataset**, I mapped airline connections. The graph was built using **NetworkX**, ensuring only active airports with connections were included. Finally, I plotted the network on an **interactive Plotly Scattermapbox**, where airports appear as **blue markers** and flight routes as **grey lines**, enabling zooming, panning, and hovering for exploration. This visualization provides a clear and interactive representation of airline connectivity across India. 

IMPORTING NECESSARY PACKAGES

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.graph_objects as go
import networkx as nx
from geopy.distance import geodesic

IMPORTING DATASETS

In [ ]:
airports = pd.read_csv(r"/kaggle/input/airlines-airport-and-routes/airports.csv")
routes = pd.read_csv(r"/kaggle/input/airlines-airport-and-routes/routes.csv")

# INDIAN AIRLINE ROUTES AND AIRPORTS

In [3]:
indian_airports = airports[airports["Country"] == "India"]
indian_airports.shape

(148, 11)

In [4]:
indian_airports = indian_airports[~indian_airports["IATA"].str.contains(r"\\N", na=False)]

In [5]:
indian_airports.shape

(121, 11)

In [6]:
indian_airports.dropna(inplace=True)

CONVERTING THE DATASETS TO GRAPH

In [7]:
# Extract Indian airport codes
indian_iata_codes = set(indian_airports["IATA"])  # Convert to set for fast lookup

# Filter routes where source OR destination is in India
indian_routes = routes[
    (routes["Source Airport"].isin(indian_iata_codes)) | 
    (routes["Destination Airport"].isin(indian_iata_codes))
]

# Initialize NetworkX Graph
G = nx.Graph()

# Add Nodes (Indian airports with latitude/longitude)
for _, row in indian_airports.iterrows():
    G.add_node(row["IATA"], pos=(row["Longitude"], row["Latitude"]))

# Add Edges (Flight routes)
for _, row in indian_routes.iterrows():
    src, dest = row["Source Airport"], row["Destination Airport"]
    if src in G and dest in G:  # Ensure both airports exist in Indian airports
        G.add_edge(src, dest)

# Print basic info about the graph
print(f"Number of Indian airports (Nodes): {G.number_of_nodes()}")
print(f"Number of flight routes (Edges): {G.number_of_edges()}")

Number of Indian airports (Nodes): 120
Number of flight routes (Edges): 198


#### From this datasets there were 120 airports in INDIA (nodes) and 198 airline routes (edges).

In [8]:
import plotly.graph_objects as go

# Extract node positions (Latitude & Longitude)
node_lats = []
node_lons = []
node_texts = []

for node, data in G.nodes(data=True):
    node_lons.append(data["pos"][0])  # Longitude
    node_lats.append(data["pos"][1])  # Latitude
    node_texts.append(node)  # IATA Code

# Extract edge positions (Routes between airports)
edge_lats = []
edge_lons = []

for edge in G.edges:
    src, dest = edge  # Source & Destination
    if src in G.nodes and dest in G.nodes:
        edge_lons.extend([G.nodes[src]["pos"][0], G.nodes[dest]["pos"][0], None])
        edge_lats.extend([G.nodes[src]["pos"][1], G.nodes[dest]["pos"][1], None])

# Create the figure
fig = go.Figure()

# Add flight routes (Edges)
fig.add_trace(go.Scattermapbox(
    lat=edge_lats,
    lon=edge_lons,
    mode="lines",
    line=dict(width=0.5, color="grey"),
    hoverinfo="none",
    showlegend=False
))

# Add airports (Nodes)
fig.add_trace(go.Scattermapbox(
    lat=node_lats,
    lon=node_lons,
    text=node_texts,
    mode="markers",
    marker=dict(size=6, color="blue"),
    hoverinfo="text",
    showlegend=False
    
))

# Update map layout
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=20.59, lon=78.96),  # Center over India
        zoom=4
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=700
)

# Show the map
fig.show(config={"scrollZoom": True})


# WORLDWIDE AIRLINE ROUTES AND AIRPORTS

In [9]:
a = set(airports["IATA"].values)
b = set(np.hstack((routes["Source Airport"].values, routes["Destination Airport"].values)))
c_airports = list(b.intersection(a))

airports = airports[airports["IATA"].isin(c_airports)][["Name", "IATA", "Latitude", "Longitude", "Altitude"]].set_index("IATA")
routes = routes[routes["Source Airport"].isin(c_airports) & routes["Destination Airport"].isin(c_airports)]

In [10]:
flight_routes = routes[["Source Airport", "Destination Airport"]].to_numpy().tolist()
node_locations = airports[["Latitude", "Longitude"]].to_dict(orient="index")

In [11]:
G = nx.DiGraph()

for airport, info in node_locations.items():
    G.add_node(airport, Latitude=info["Latitude"], Longitude=info["Longitude"])

for route in flight_routes:
    source, destination = route
    G.add_edge(source, destination)


In [18]:
all_lats = []
all_lons = []
all_texts = []

edges = list(G.edges)
nodes = dict(G.nodes.data())

for route in edges:
    all_lats.extend([nodes[route[0]]["Latitude"], nodes[route[1]]["Latitude"], None])
    all_lons.extend([nodes[route[0]]["Longitude"], nodes[route[1]]["Longitude"], None])
    all_texts.extend([route[0], route[1], None])

fig = go.Figure(go.Scattermapbox(
    lat=all_lats,
    lon=all_lons,
    text=all_texts,
    mode="lines+markers",
    marker=dict(size=4, color="blue"),
    line=dict(width=0.05, color="grey")
    )
)

fig.update_layout(
    margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
    mapbox=dict(
        style="open-street-map",
        center={'lon': 0, 'lat': 20},  
        zoom=0,  
        bounds={"west": -180, "east": 180, "south": -85, "north": 85}
    ),
    height = 700
)

fig.show(config={"scrollZoom": True})

#### I know it has a lot of edges which is harder get inference from denser areas. This plot is interactive so you could easily zoom and get an idea 

#### about the airline network data. Scroll down to see about **centralities**.

In [13]:
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)

top_5_deg_centrality = sorted(degree_centrality, key=lambda x: degree_centrality[x], reverse=True)[:5]
top_5_bet_centrality = sorted(betweenness_centrality, key=lambda x: betweenness_centrality[x], reverse=True)[:5]
top_5_clo_centrality = sorted(closeness_centrality, key=lambda x: closeness_centrality[x], reverse=True)[:5]
top_5_eig_centrality = sorted(eigenvector_centrality, key=lambda x: eigenvector_centrality[x], reverse=True)[:5]

In [14]:
# Extract nodes and edges for top 5 degree centrality airports
top_deg_nodes = top_5_deg_centrality
top_deg_edges = [(u, v) for u, v in G.edges if u in top_deg_nodes or v in top_deg_nodes]

# Store positions
top_lats = []
top_lons = []
top_texts = []

edge_lats = []
edge_lons = []

# Extract edge coordinates
for src, dest in top_deg_edges:
    edge_lats.extend([G.nodes[src]["Latitude"], G.nodes[dest]["Latitude"], None])
    edge_lons.extend([G.nodes[src]["Longitude"], G.nodes[dest]["Longitude"], None])

# Extract node coordinates
for node in top_deg_nodes:
    top_lats.append(G.nodes[node]["Latitude"])
    top_lons.append(G.nodes[node]["Longitude"])
    top_texts.append(node)

# Create the plot
fig = go.Figure()

# Add flight routes (Edges)
fig.add_trace(go.Scattermapbox(
    lat=edge_lats,
    lon=edge_lons,
    mode="lines",
    line=dict(width=0.3, color="grey"),
    hoverinfo="none",
    showlegend=False
))

# Add top airports (Nodes)
fig.add_trace(go.Scattermapbox(
    lat=top_lats,
    lon=top_lons,
    text=top_texts,
    mode="markers+text",
    marker=dict(size=8, color="red"),
    textfont=dict(size=10, color="black"),  # Customize text appearance
    textposition="top center",
    hoverinfo="text",
    showlegend=False
))

# Update layout
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center={'lon': 0, 'lat': 20},  
        zoom=2,
        bounds={"west": -180, "east": 180, "south": -85, "north": 85}

    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=700,
    showlegend=False
)

# Show the map
fig.show(config={"scrollZoom": True})


#### Top 5 airports with highest degree centrality

FRA: Frankfurt Airport (Frankfurt am Main, Germany)

CDG: Charles de Gaulle Airport (Paris, France)

AMS: Amsterdam Airport Schiphol (Amsterdam, Netherlands)

IST: Istanbul Airport (Istanbul, Turkey)

ATL: Hartsfield–Jackson Atlanta International Airport (Atlanta, Georgia, USA)

In [15]:
# Extract nodes and edges for top 5 degree centrality airports
top_deg_nodes = top_5_bet_centrality
top_deg_edges = [(u, v) for u, v in G.edges if u in top_deg_nodes or v in top_deg_nodes]

# Store positions
top_lats = []
top_lons = []
top_texts = []

edge_lats = []
edge_lons = []

# Extract edge coordinates
for src, dest in top_deg_edges:
    edge_lats.extend([G.nodes[src]["Latitude"], G.nodes[dest]["Latitude"], None])
    edge_lons.extend([G.nodes[src]["Longitude"], G.nodes[dest]["Longitude"], None])

# Extract node coordinates
for node in top_deg_nodes:
    top_lats.append(G.nodes[node]["Latitude"])
    top_lons.append(G.nodes[node]["Longitude"])
    top_texts.append(node)

# Create the plot
fig = go.Figure()

# Add flight routes (Edges)
fig.add_trace(go.Scattermapbox(
    lat=edge_lats,
    lon=edge_lons,
    mode="lines",
    line=dict(width=0.3, color="grey"),
    hoverinfo="none",
    showlegend=False
))

# Add top airports (Nodes)
fig.add_trace(go.Scattermapbox(
    lat=top_lats,
    lon=top_lons,
    text=top_texts,
    mode="markers+text",
    marker=dict(size=8, color="red"),
    textfont=dict(size=10, color="black"),  # Customize text appearance
    textposition="top center",
    hoverinfo="text",
    showlegend=False
))

# Update layout
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        center={'lon': 0, 'lat': 20},  
        zoom=2,
        bounds={"west": -180, "east": 180, "south": -85, "north": 85}

    ),
    margin=dict(l=0, r=0, t=0, b=0),
    height=700,
    showlegend=False
)

# Show the map
fig.show(config={"scrollZoom": True})


#### Top 5 airports with highest betweenes centrality

CDG: Charles de Gaulle Airport (Paris, France)

LAX: Los Angeles International Airport (Los Angeles, USA)

DXB: Dubai International Airport (Dubai, UAE)

ANC: Ted Stevens Anchorage International Airport (Anchorage, USA)

FRA: Frankfurt Airport (Frankfurt am Main, Germany)

In [17]:
source_iata = input("Enter source airport IATA code: ").strip().upper()
destination_iata = input("Enter destination airport IATA code: ").strip().upper()

# Check if nodes exist in the graph
if source_iata in G.nodes and destination_iata in G.nodes:
    # Compute direct great-circle distance between source and destination
    src_coords = (G.nodes[source_iata]["Latitude"], G.nodes[source_iata]["Longitude"])
    dest_coords = (G.nodes[destination_iata]["Latitude"], G.nodes[destination_iata]["Longitude"])
    direct_distance = geodesic(src_coords, dest_coords).kilometers

    # Find all possible routes with a maximum of one stop
    possible_routes = list(nx.all_simple_paths(G, source=source_iata, target=destination_iata, cutoff=2))

    # Filter routes: Remove any route exceeding 2× direct distance
    valid_routes = []
    for path in possible_routes:
        total_route_distance = sum(
            geodesic((G.nodes[path[i]]["Latitude"], G.nodes[path[i]]["Longitude"]),
                     (G.nodes[path[i+1]]["Latitude"], G.nodes[path[i+1]]["Longitude"])).kilometers
            for i in range(len(path) - 1)
        )
        
        # Only keep routes that are within the allowed distance
        if total_route_distance <= 2 * direct_distance:
            valid_routes.append(path)

    # If no valid routes, exit early
    if not valid_routes:
        print("No efficient routes found within the 2× distance constraint.")
    else:
        # Lists to store edge data
        edge_lats_direct, edge_lons_direct = [], []  # Direct (fastest) routes (Green)
        edge_lats_indirect, edge_lons_indirect = [], []  # Indirect routes (Blue)
        node_lats, node_lons, node_texts, node_colors = [], [], [], []

        # Process valid routes
        for path in valid_routes:
            full_route_lats, full_route_lons = [], []
            for i in range(len(path) - 1):
                lat1, lon1 = G.nodes[path[i]]["Latitude"], G.nodes[path[i]]["Longitude"]
                lat2, lon2 = G.nodes[path[i + 1]]["Latitude"], G.nodes[path[i + 1]]["Longitude"]
                
                full_route_lats.extend([lat1, lat2, None])
                full_route_lons.extend([lon1, lon2, None])

            # Direct routes (fastest) → Green edges
            if len(path) == 2:
                edge_lats_direct.extend(full_route_lats)
                edge_lons_direct.extend(full_route_lons)
            else:
                edge_lats_indirect.extend(full_route_lats)
                edge_lons_indirect.extend(full_route_lons)

        # Collect source and destination information
        node_lats.extend([G.nodes[source_iata]["Latitude"], G.nodes[destination_iata]["Latitude"]])
        node_lons.extend([G.nodes[source_iata]["Longitude"], G.nodes[destination_iata]["Longitude"]])
        node_texts.extend([source_iata, destination_iata])
        node_colors.extend(["red", "orange"])  # Red for source, Orange for destination

        # Create the Plotly map
        fig = go.Figure()

        # Add indirect flight routes (Blue edges - possible routes)
        fig.add_trace(go.Scattermapbox(
            lat=edge_lats_indirect,
            lon=edge_lons_indirect,
            mode="lines",
            line=dict(width=1, color="blue"),
            hoverinfo="none",
            name="<b>Possible Routes</b>"
        ))

        # Add direct flight routes (Green edges - fastest routes)
        fig.add_trace(go.Scattermapbox(
            lat=edge_lats_direct,
            lon=edge_lons_direct,
            mode="lines",
            line=dict(width=3, color="green"),
            hoverinfo="none",
            name="<b>Direct Route (Fastest Route)</b>"
        ))

        # Add source and destination airports (Nodes)
        fig.add_trace(go.Scattermapbox(
            lat=node_lats,
            lon=node_lons,
            text=node_texts,
            mode="markers+text",
            marker=dict(size=10, color=node_colors),
            textfont=dict(size=12, color="black"),
            textposition="top center",
            hoverinfo="text",
            showlegend=False  # **Hide extra legend entry**
        ))

        # Add custom legend manually for nodes
        for color, label in zip(["red", "orange"], ["Source Airport", "Destination Airport"]):
            fig.add_trace(go.Scattermapbox(
                lat=[None], lon=[None],  # Dummy entry for legend
                mode="markers",
                marker=dict(size=10, color=color),
                name=f"<b>{label}</b>"
            ))

        fig.update_layout(
            mapbox=dict(
                style="open-street-map",
                center={'lon': (G.nodes[source_iata]["Longitude"] + G.nodes[destination_iata]["Longitude"]) / 2,
                        'lat': (G.nodes[source_iata]["Latitude"] + G.nodes[destination_iata]["Latitude"]) / 2},
                zoom=3
            ),
            margin=dict(l=0, r=0, t=0, b=0),
            height=700,
            legend=dict(
                x=0, y=1,
                font=dict(size=14)  # Make legend text bold
            )
        )

        # Show the map
        fig.show(config={"scrollZoom": True})

else:
    print("Invalid IATA codes or airports not in the dataset.")


Enter source airport IATA code:  maa
Enter destination airport IATA code:  dxb


#### Enter the source and destination airports to get routes connected to it.